In [ ]:
reset -sf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r /content/Math_solving_using_LLM

In [ ]:
!git clone https://github.com/dunp7/Math_solving_using_LLM.git

Cloning into 'Math_solving_using_LLM'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 311 (delta 59), reused 62 (delta 28), pack-reused 216 (from 1)
Receiving objects: 100% (311/311), 979.56 KiB | 6.85 MiB/s, done.
Resolving deltas: 100% (212/212), done.


In [ ]:
!pip install huggingface_hub
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install -r /content/Math_solving_using_LLM/requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu124


# Libraries

In [ ]:
import random
import torch
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from huggingface_hub import login
import sys
import os
sys.path.append("/content/Math_solving_using_LLM")
sys.path.append("/content/Math_solving_using_LLM/my_utils")


from generate_results import generate_answers, generate_labels
from my_utils.data import load_ds, sample_ds, load_results
from huggingface_hub import login

token_comp = "your_hugging_face_token"
login(token= token_comp)


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
seed = 42
random.seed(seed)

cuda


# Load datasets

In [ ]:
n_samples = 100
_, svamp_val = load_ds("svamp", seed)
svamp_sample = sample_ds(svamp_val, n_samples, seed, "svamp")

del svamp_val
datasets = [svamp_sample]

Dataset: svamp
Dataset({
    features: ['ID', 'Body', 'Question', 'Equation', 'Answer', 'Type', 'question_concat', 'question', 'type', 'equation', 'id', 'answers'],
    num_rows: 100
}) 



In [ ]:
datasets[0]['question'][0] # How many more cookies did he eat than those he gave to his friend?

'Paco had 17 cookies. He ate 14 cookies and gave 13 of them to his friend. How many more cookies did he eat than those he gave to his friend?'

# Generate answers

In [ ]:
# LLM - Falcon_instruct(7B), Mistral(7B, Instruct)

### Generate answers LLM
models = {
    "gemma2_9b": "google/gemma-2-9b-it",
    "gemma2_2b": "google/gemma-2-2b-it",
    "llama2_7b": "meta-llama/Llama-2-7b-chat-hf",
    "mistral7b_instruct": "mistralai/Mistral-7B-Instruct-v0.2",
}

In [ ]:
# Prompt
# Prompts
prompts = {
    "default": None,
    "zero_shot_CoT": "Let’s think step by step.",
    "ite_ape": "We’ll approach this methodically, proceeding one step at a time.",
    # "apo": "Carefully analyze the details and perform precise arithmetic operations step by step, ensuring to apply the correct mathematical principles for accurate calculations.",
    # "pe2": "Carefully calculate step by step, considering all details, including unique counts and overlaps. Accurately apply arithmetic to find the numerical answer, ensuring logical operations are correctly followed."
}


## Gen answers

In [ ]:
import shutil
import os
def save_to_google_drive(source_folder, destination_folder, model_name):
    """
    Saves the given source folder to the specified destination in Google Drive under a specific model name.
    """
    # Create destination folder for hallucination_data
    hallucination_data_folder = os.path.join(destination_folder, 'hallucination_data')
    os.makedirs(hallucination_data_folder, exist_ok=True)

    # Create destination folder for the specific model
    model_folder = os.path.join(hallucination_data_folder, model_name)
    try:
        if os.path.exists(model_folder):
            shutil.rmtree(model_folder)  # Clear existing model folder
        shutil.copytree(source_folder, model_folder)
        print(f"Data for {model_name} copied to {model_folder}")
    except Exception as e:
        print(f"Error copying folder for {model_name} to Google Drive: {e}")

In [ ]:
# Generate answers
for model_name, model_path in models.items():
    print(f"Processing model: {model_name}...")

    # Load model and tokenizer
    gen_model = AutoModelForCausalLM.from_pretrained(
        model_path, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    gen_tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Iterate over prompts
    for prompt_name, chosen_prompt in prompts.items():
        print(f"  Using prompt: {prompt_name}")

        # Create output directory
        data_answer_path = f"data/{model_name}/{prompt_name}/"
        os.makedirs(data_answer_path, exist_ok=True)

        # Generate answers (ensure this function is defined)
        generate_answers(datasets, data_answer_path, gen_model, gen_tokenizer, chosen_prompt)

    # Save the processed data to Google Drive
    save_to_google_drive(f"/content/data/{model_name}", "/content/drive/MyDrive/Math Solving Problem", model_name)

    # Clean up memory
    del gen_model, gen_tokenizer
    torch.cuda.empty_cache()

Processing model: mistral7b_instruct...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  Using prompt: zero_shot_CoT

Generating responses for svamp dataset...


Generating Answers:   0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
